In [1]:
import rasterio
import numpy as np

from rural_beauty.config import data_dir, CLC_coverage_EU_dir, streets_EU_raster, powerlines_EU_raster



This is the rural_beauty module


In [ ]:
coefs = {
    'dem_1': 0.009,
    'dem_3': 0.004,
    'seemee_1': 0.011, 
}

In [3]:
features = {
 'dem_1'    : data_dir / 'cleaned/dem/DEM_EU_range.tif',
 'dem_3'    : data_dir / 'cleaned/dem/neighborhood/DEM_EU_range_zone3.tif',
 'seemee_1' : CLC_coverage_EU_dir / 'code_seemee.tif',
 'heide_1'  : CLC_coverage_EU_dir / 'code_heide.tif',
 'natgru_1_2' : CLC_coverage_EU_dir / 'neighborhood/code_natgru_zone1_2.tif',
 'wein_1'   : CLC_coverage_EU_dir / 'code_wein.tif', 
 'acker_1_2': CLC_coverage_EU_dir / 'neighborhood/code_acker_zone1_2.tif',
 'stoer_1_2'  :CLC_coverage_EU_dir / 'neighborhood/code_stoer_zone1_2.tif',
 'stra_1' : data_dir / 'cleaned/osm/len_streets_EU_4647.tif', 
 'leit_1'   : data_dir / 'cleaned/osm/len_powerlines_EU_4647.tif', 
           }


In [7]:
constant = 4.595

coefs = {
  'dem_1'      : 0.009,
  'dem_3'      : 0.004,
  'seemee_1'   : 0.011,
  'heide_1'    : 0.035,
  'natgru_1_2' : 0.023,
  'wein_1'     : 0.009,
  'acker_1_2'  :-0.008,
  'stoer_1_2'  :-0.024,
  'stra_1'     :-0.00002711,
  'leit_1'     :-0.00006803,
}



In [14]:
raster_paths = [v for k,v in features.items()]
coefficients = [coefs[k] for k,v in features.items()]
raster_paths

[PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/DEM_EU_range.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone3.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_seemee.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_heide.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_natgru_zone1_2.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_wein.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_acker_zone1_2.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_stoer_zone1_2.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/osm/len_streets_EU_4647.ti

In [17]:
# New base directory for the adjusted rasters
new_base_dir =  data_dir / 'forprediction/uniqueness'

new_base_dir.mkdir(parents=True, exist_ok=True)

# Create new list of file paths for the adjusted rasters
adjusted_raster_paths = [new_base_dir / raster_path.name for raster_path in raster_paths]


In [27]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
import numpy as np

def subset_and_align_rasters(raster_paths, output_paths):
    """
    Subset all rasters to the largest area contained within all inputs, 
    and align them to the same resolution and extent.

    Args:
    - raster_paths (list of str): List of file paths to the input rasters.
    - output_paths (list of str): List of output file paths to save the aligned rasters.
    """
    # Open all rasters and get their bounds
    bounds_list = []
    transforms = []
    crs = None

    for path in raster_paths:
        with rasterio.open(path) as src:
            bounds_list.append(src.bounds)
            transforms.append(src.transform)
            if crs is None:
                crs = src.crs  # Assuming all rasters are in the same CRS, else reproject would be needed.

    # Calculate the largest common intersection (extent)
    intersection_bounds = (
        max([b.left for b in bounds_list]),   # Left bound
        max([b.bottom for b in bounds_list]), # Bottom bound
        min([b.right for b in bounds_list]),  # Right bound
        min([b.top for b in bounds_list])     # Top bound
    )

    ref_resolution = 1000
    pixel_width = ref_resolution
    pixel_height = ref_resolution
    width = int((intersection_bounds[2] - intersection_bounds[0]) / pixel_width)
    height = int((intersection_bounds[3] - intersection_bounds[1]) / pixel_height)

    # Align rasters to this common extent
    for idx, (raster_path, output_path) in enumerate(zip(raster_paths, output_paths)):
        with rasterio.open(raster_path) as src:
            # Calculate the transform for the common extent
            transform, _, _ = calculate_default_transform(
                src.crs, src.crs, width=width, height=height, 
                left=intersection_bounds[0], bottom=intersection_bounds[1], 
                right=intersection_bounds[2], top=intersection_bounds[3])

            # Create a new dataset with the common extent
            kwargs = src.meta.copy()
            kwargs.update({
                'height': height,
                'width': width,
                'transform': transform
            })
            print(f"Aligning {raster_path}. dimensions: {width}, {height}")

            # Read the data and reproject to align with the common extent
            with rasterio.open(output_path, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):  # Reproject all bands
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.nearest  # You can change this to another method if needed
                    )





In [28]:

subset_and_align_rasters(raster_paths, adjusted_raster_paths)

Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/DEM_EU_range.tif. dimensions: 6564, 4058
| 1000.17, 0.00, 29172876.76|
| 0.00,-1000.17, 7986389.04|
| 0.00, 0.00, 1.00|


Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone3.tif. dimensions: 6564, 4058
| 1000.17, 0.00, 29172876.76|
| 0.00,-1000.17, 7986389.04|
| 0.00, 0.00, 1.00|
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_seemee.tif. dimensions: 6564, 4058
| 1000.17, 0.00, 29172876.76|
| 0.00,-1000.17, 7986389.04|
| 0.00, 0.00, 1.00|
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_heide.tif. dimensions: 6564, 4058
| 1000.17, 0.00, 29172876.76|
| 0.00,-1000.17, 7986389.04|
| 0.00, 0.00, 1.00|
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_natgru_zone1_2.tif. dimensions: 6564, 4058
| 1000.17, 0.00, 29172876.76|
| 0.00,-1000.17, 7986389.04|
| 0.00, 0.00, 1.00|
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_wein.tif. dimensions: 6564, 4058
| 1000.17, 0.00, 29172876.76|
| 0.

In [31]:
import rasterio
import numpy as np

def linear_combination_rasters(raster_paths, coefficients, constant, output_path):
    """
    Create a new raster from a linear combination of input rasters.
    
    Args:
    - raster_paths (list of str): List of file paths to the input rasters.
    - coefficients (list of float): Coefficients for the linear combination, same length as raster_paths.
    - output_path (str): File path to save the output raster.
    """
    # Open the first raster to get metadata
    with rasterio.open(raster_paths[0]) as src:
        meta = src.meta
        data = src.read(1) * coefficients[0]  # Read the first raster and multiply by its coefficient

    # Loop through the remaining rasters and apply the linear combination
    for i, raster_path in enumerate(raster_paths[1:], start=1):
        with rasterio.open(raster_path) as src:
            data += src.read(1) * coefficients[i]  # Add the weighted raster data

    data += constant
    # Update metadata if needed (e.g., datatype)
    meta.update(dtype=rasterio.float64)

    # Save the result as a new raster
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(data.astype(rasterio.float64), 1)




In [34]:
linear_combination_rasters(adjusted_raster_paths, coefficients, constant, data_dir / 'testoutput.tif')